In [ ]:
import numpy as np
import matplotlib.pyplot as  plt
import os

from datetime import datetime
import h5py

# from Tools.plot import plot
from multiprocessing import Pool, cpu_count

from qTools import *

# Define Parameters

In [ ]:
wC = 1.
# wQ: sweepable
# g: sweepable

quench = 5/wC
tau = 10e-3

steps = int(quench/tau)
print(steps)

# Define sweep lists

In [ ]:
wQ_wC = np.linspace(10, 50, 20)
wQ = wQ_wC * wC

g_gc = np.linspace(0, 2, steps+1)

g = []
g_ratio = []
for wQi in wQ:
    gc = np.sqrt(wQi*wC)/2
    g.append(g_gc*gc)
    g_ratio.append(gc)

# Create System

In [ ]:
qub = Qubit(frequency=0)
cav = Cavity(dimension=60, frequency=wC/2)
system = qub + cav
coupling = system.JC(0)
coupling.superSys = system

# Create Protocol

In [ ]:
protocol = qProtocol(superSys=system)
JC = freeEvolution(ratio=0.5, superSys=system)
AJC = freeEvolution(superSys=system)
qubset = JC.createUpdate(system=qub, key='frequency', value=0)
X = xGate(implementation='instant', superSys=qub)
protocol.addStep(JC, X, AJC ,X, JC)

# Define Simulation & Sweeps

In [ ]:
qubset.name

In [ ]:
system.simStepSize = tau
system.simTotalTime = quench
system.simDelStates = True
system.initialState = [1,0]

system.simulation.addProtocol(protocol=protocol, system=system, protocolRemove=system._freeEvol)

"""def gAtT(coupling, time):
    wQi = coupling.getObjByName('Update1').value
    gc = np.sqrt(wQi)/2
    n = int(time/tau)
    return g_gc[n]*gc"""

def gAtT(coupling, time):
    n = int(time/tau)
    return g_gc[n]*coupling.auxObj.ratio

coupling.timeDependency = gAtT

wQSweep = system.simulation.Sweep.createSweep(system=qubset, sweepKey='value', sweepList=wQ)
system.simulation.Sweep.createSweep(system=qubset.auxObj, sweepKey='ratio', sweepList=g_ratio)

# Define results function

In [ ]:
n = cav.freeMat
def expect(qSim, state):
    qSim.qRes.results['cavParity'].append(expectationKet(n, state))
    qSim.qRes.results['g'].append(qSim.getObjByName('JCcoupling').couplingStrength)

system.simCompute = expect

system.runSimulation(p=True, coreCount='all')

In [ ]:
sim = Simulation()
sim.stepSize = tau
system.initialState = [1,0]
sim.delStates = True
sim.addQSystems(system, protocol)
gSweep = sim.timeDependency.createSweep(system=coupling, sweepKey='couplingStrength')
wQSweep = sim.Sweep.createSweep(system=qubset, sweepKey='value', sweepList=wQ)
asdasd = sim.Sweep.createSweep(system=gSweep, sweepKey='sweepList', sweepList=g)

# Run simulation

In [ ]:
n = cav.freeMat
def expect(qSim, state):
    qSim.qRes.results['cavParity'].append(expectationKet(n, state))
    qSim.qRes.results['g'].append(qSim.getObjByName('qCoupling1'))

sim.compute = expect

sim.run(p=True, coreCount='all')

# Plot

In [ ]:
norm = []
ones = np.ones(g_gc.size)
for wQi in wQ:
    norm.append(ones * cav.frequency/wQi)
norm = np.array(norm)
    
Y, X = np.meshgrid(g_gc, wQ)
fig = plt.figure()
surf = plt.pcolormesh(Y, X, norm*system.simulation.qRes.results['cavParity'])
plt.colorbar(surf)
plt.xlabel('g/gc')
plt.ylabel(r'omegaQ')
plt.title('Scalled photon number')

plt.show()

In [ ]:
g_gc = np.linspace(0, 2, steps+2)

In [ ]:
norm = []
ones = np.ones(g_gc.size)
for wQi in wQ:
    norm.append(ones * cav.frequency/wQi)
norm = np.array(norm)
    
Y, X = np.meshgrid(g_gc, wQ)
fig = plt.figure()
surf = plt.pcolormesh(Y, X, norm*sim.qRes.results['cavParity'])
plt.colorbar(surf)
plt.xlabel('g/gc')
plt.ylabel(r'omegaQ')
plt.title('Scalled photon number')

plt.show()